Домашнее задание:

- К вам приходит руководитель продукта “Интернет-магазин” 
- И спрашивает про удержание клиентов и повторные покупки
- У него есть опасение, что из-за проблем с доставкой товаров в магазине у нас стали больше оттекать пользователи
- Он просит вас сравнить, на сколько сейчас все хуже, чем в прошлом году? 

Данные лежат в папке занятия на сервере:

- есть транзакции (покупки) в интернет магазине, в том числе id клиента, id покупки, timestamp покупки и тп
- orders.csv, customers.csv

Ваши задачи: 

1. Проверить гипотезу руководителя продукта 
2. Дать руководителю продукта инструмент для самостоятельного принятия решений по качеству удержания пользовательской базы

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
df_orders = pd.read_csv('orders.csv', parse_dates=['order_date'])
df_orders['order_month'] = df_orders['order_date'].dt.to_period('M')
df_orders['order_year'] = df_orders['order_date'].dt.to_period('Y')

In [3]:
df_orders.head()

,id,order_date,ship_mode,customer_id,sales,order_month,order_year
0,100006,2014-09-07,Standard,DK-13375,377.970,2014-09,2014
1,100090,2014-07-08,Standard,EB-13705,699.192,2014-07,2014
2,100293,2014-03-14,Standard,NF-18475,91.056,2014-03,2014
3,100328,2014-01-28,Standard,JC-15340,3.928,2014-01,2014
4,100363,2014-04-08,Standard,JM-15655,21.376,2014-04,2014


In [4]:
df_customers = pd.read_csv('customers.csv')

In [5]:
df_customers.head()

,id,name,segment,state,city
0,CG-12520,Claire Gute,Consumer,Kentucky,Henderson
1,DV-13045,Darrin Van Huff,Corporate,California,Los Angeles
2,SO-20335,Sean O'Donnell,Consumer,Florida,Fort Lauderdale
3,BH-11710,Brosina Hoffman,Consumer,California,Los Angeles
4,AA-10480,Andrew Allen,Consumer,North Carolina,Concord


In [6]:
df_merge = pd.merge(df_orders, df_customers, how='inner', left_on='customer_id', right_on='id', suffixes=('', '_y'))
df_merge.drop(columns=['id_y'], inplace=True)

In [7]:
df_merge.head()

,id,order_date,ship_mode,customer_id,sales,order_month,order_year,name,segment,state,city
0,100006,2014-09-07,Standard,DK-13375,377.970,2014-09,2014,Dennis Kane,Consumer,Ohio,Marion
1,131884,2015-12-06,Same Day,DK-13375,594.002,2015-12,2015,Dennis Kane,Consumer,Ohio,Marion
2,145065,2015-12-12,First,DK-13375,32.308,2015-12,2015,Dennis Kane,Consumer,Ohio,Marion
3,133046,2017-07-27,Second,DK-13375,297.990,2017-07,2017,Dennis Kane,Consumer,Ohio,Marion
4,165099,2017-12-11,First,DK-13375,1.392,2017-12,2017,Dennis Kane,Consumer,Ohio,Marion


In [8]:
df_first = df_merge.groupby('customer_id')['order_year'].min().reset_index()
df_first.columns = ['customer_id', 'first_order']

In [9]:
df_first.head()

,customer_id,first_order
0,AA-10315,2014
1,AA-10375,2014
2,AA-10480,2014
3,AA-10645,2014
4,AB-10015,2014


In [10]:
df_full = pd.merge(df_merge, df_first, how='inner', on='customer_id')

In [11]:
df_full.head()

,id,order_date,ship_mode,customer_id,sales,order_month,order_year,name,segment,state,city,first_order
0,100006,2014-09-07,Standard,DK-13375,377.970,2014-09,2014,Dennis Kane,Consumer,Ohio,Marion,2014
1,131884,2015-12-06,Same Day,DK-13375,594.002,2015-12,2015,Dennis Kane,Consumer,Ohio,Marion,2014
2,145065,2015-12-12,First,DK-13375,32.308,2015-12,2015,Dennis Kane,Consumer,Ohio,Marion,2014
3,133046,2017-07-27,Second,DK-13375,297.990,2017-07,2017,Dennis Kane,Consumer,Ohio,Marion,2014
4,165099,2017-12-11,First,DK-13375,1.392,2017-12,2017,Dennis Kane,Consumer,Ohio,Marion,2014


In [12]:
df_users = df_full.groupby(['customer_id', 'first_order', 'order_year', 'segment', 'ship_mode'])\
    .agg({'sales':'sum', 'id':'count'}).reset_index()

In [13]:
df_users.head()

,customer_id,first_order,order_year,segment,ship_mode,sales,id
0,AA-10315,2014,2014,Consumer,Standard,756.048,2
1,AA-10315,2014,2015,Consumer,First,26.960,1
2,AA-10315,2014,2016,Consumer,Standard,4406.072,1
3,AA-10315,2014,2017,Consumer,Standard,374.480,1
4,AA-10375,2014,2014,Consumer,Standard,50.792,2


In [14]:
def get_crosstab(df, value_column):
    df_crosstab = pd.crosstab(df.customer_id, df.order_year, values=df[value_column], aggfunc=sum)\
        .reset_index()
    df_crosstab = pd.merge(df_crosstab, df_first, how='inner', on='customer_id')
    del df_crosstab['customer_id']
    return df_crosstab

In [15]:
def get_retention(df):
    df_triangle = df.groupby(['first_order']).sum().reset_index()
    df_triangle['Base'] = np.diag(df_triangle.iloc[:,1:])
    df_retention = pd.DataFrame(df_triangle[['first_order', 'Base']])
    df_retention.columns = ['Cohort', 'Base']

    for col in df_triangle.columns[1:-1]:
        df_retention[col] = round(df_triangle[col] / df_triangle['Base'], 3)

    for i in range(2, len(df_retention.columns)):
        df_retention.iloc[i-1:, i] = None
        
    return df_retention

### Sales retention

In [16]:
get_retention(get_crosstab(df_users, 'sales'))

,Cohort,Base,2014,2015,2016,2017
0,2014,484247.4981,1.0,0.754,0.939,1.101
1,2015,105388.4962,NaN,1.000,0.944,1.250
2,2016,54803.6200,NaN,NaN,1.000,1.111
3,2017,7511.8050,NaN,NaN,NaN,1.000


### Transaction retention

In [17]:
get_retention(get_crosstab(df_users, 'id'))

,Cohort,Base,2014,2015,2016,2017
0,2014,969.0,1.0,0.833,1.028,1.288
1,2015,231.0,NaN,1.000,0.931,1.268
2,2016,104.0,NaN,NaN,1.000,1.221
3,2017,19.0,NaN,NaN,NaN,1.000


### Sales retention by segment

In [18]:
for segment in df_users.segment.unique():
    print(f'\nSegment: {segment}')
    print(get_retention(get_crosstab(df_users[df_users.segment==segment], 'sales')))


Segment: Consumer
  Cohort         Base  2014   2015   2016   2017
0   2014  266096.8126   1.0  0.789  0.818  0.906
1   2015   56590.8551   NaN  1.000  0.832  0.817
2   2016   32119.4550   NaN    NaN  1.000  1.229
3   2017    4985.0760   NaN    NaN    NaN  1.000

Segment: Home Office
  Cohort        Base  2014   2015   2016   2017
0   2014  89715.8118   1.0  0.616  0.874  1.296
1   2015  19942.1216   NaN  1.000  1.120  1.475
2   2016   4483.5420   NaN    NaN  1.000  2.905
3   2017    729.6480   NaN    NaN    NaN  1.000

Segment: Corporate
  Cohort         Base  2014   2015   2016   2017
0   2014  128434.8737   1.0  0.778  1.237  1.367
1   2015   28855.5195   NaN  1.000  1.043  1.944
2   2016   18200.6230   NaN    NaN  1.000  0.462
3   2017    1797.0810   NaN    NaN    NaN  1.000


### Sales retention by ship_mode

In [19]:
for ship_mode in df_users.ship_mode.unique():
    print(f'\nShip_mode: {ship_mode}')
    print(get_retention(get_crosstab(df_users[df_users.ship_mode==ship_mode], 'sales')))


Ship_mode: Standard
  Cohort         Base  2014   2015   2016   2017
0   2014  305621.3196   1.0  0.716  0.921  0.935
1   2015   65723.8550   NaN  1.000  0.791  1.142
2   2016   38855.4240   NaN    NaN  1.000  0.752
3   2017    5644.8960   NaN    NaN    NaN  1.000

Ship_mode: First
  Cohort        Base  2014   2015   2016   2017
0   2014  59769.2645   1.0  0.997  1.010  1.590
1   2015   9644.0606   NaN  1.000  1.632  2.143
2   2016   6153.2850   NaN    NaN  1.000  3.814
3   2017    955.7840   NaN    NaN    NaN  1.000

Ship_mode: Second
  Cohort         Base  2014   2015   2016   2017
0   2014  101386.7800   1.0  0.642  0.811  1.105
1   2015   23970.2056   NaN  1.000  1.180  1.216
2   2016    9537.8710   NaN    NaN  1.000  0.697
3   2017     911.1250   NaN    NaN    NaN  1.000

Ship_mode: Same Day
  Cohort       Base  2014   2015   2016   2017
0   2014  17470.134   1.0  1.234  1.759  2.308
1   2015   6050.375   NaN  1.000  0.581  1.136
2   2016    257.040   NaN    NaN  1.000  6.163


### Transaction retention by segment

In [20]:
for segment in df_users.segment.unique():
    print(f'\nSegment: {segment}')
    print(get_retention(get_crosstab(df_users[df_users.segment==segment], 'id')))


Segment: Consumer
  Cohort   Base  2014   2015   2016   2017
0   2014  524.0   1.0  0.817  0.954  1.265
1   2015  107.0   NaN  1.000  0.869  1.215
2   2016   58.0   NaN    NaN  1.000  1.207
3   2017   13.0   NaN    NaN    NaN  1.000

Segment: Home Office
  Cohort   Base  2014   2015   2016   2017
0   2014  158.0   1.0  0.899  1.095  1.405
1   2015   49.0   NaN  1.000  1.122  1.429
2   2016   14.0   NaN    NaN  1.000  1.786
3   2017    1.0   NaN    NaN    NaN  1.000

Segment: Corporate
  Cohort   Base  2014   2015   2016   2017
0   2014  287.0   1.0  0.826  1.125  1.265
1   2015   75.0   NaN  1.000  0.893  1.240
2   2016   32.0   NaN    NaN  1.000  1.000
3   2017    5.0   NaN    NaN    NaN  1.000


### Transaction retention by ship_mode

In [21]:
for ship_mode in df_users.ship_mode.unique():
    print(f'\nShip_mode: {ship_mode}')
    print(get_retention(get_crosstab(df_users[df_users.ship_mode==ship_mode], 'id')))


Ship_mode: Standard
  Cohort   Base  2014   2015   2016   2017
0   2014  586.0   1.0  0.845  1.002  1.229
1   2015  141.0   NaN  1.000  0.943  1.284
2   2016   62.0   NaN    NaN  1.000  1.242
3   2017   12.0   NaN    NaN    NaN  1.000

Ship_mode: First
  Cohort   Base  2014   2015   2016   2017
0   2014  145.0   1.0  0.772  1.103  1.448
1   2015   31.0   NaN  1.000  1.194  1.516
2   2016   18.0   NaN    NaN  1.000  1.278
3   2017    4.0   NaN    NaN    NaN  1.000

Ship_mode: Second
  Cohort   Base  2014   2015   2016   2017
0   2014  190.0   1.0  0.826  0.995  1.311
1   2015   49.0   NaN  1.000  0.673  1.020
2   2016   22.0   NaN    NaN  1.000  1.000
3   2017    3.0   NaN    NaN    NaN  1.000

Ship_mode: Same Day
  Cohort  Base  2014   2015  2016   2017
0   2014  48.0   1.0  0.896  1.25  1.438
1   2015  10.0   NaN  1.000  1.20  1.500
2   2016   2.0   NaN    NaN  1.00  2.500


## Выводы
Если сравнивать ретеншены 2017 и 2016 годов, то:
1. Тотально sales и transaction ретеншены 2017 лучше 2016
2. Но есть слабые когорты, где хуже: 
 - Для sales-retention это:
    - Segment: Corporate, first order=2016
    - Ship_mode: Standard, first order=2016; Second, first order=2016
 - Для transaction-retention слабые когорты не наблюдаются
3. Нельзя не отметить сокращение когорт по годам, но это не влияет на удержание клиентов.